# debug-SEsystem-enhancedprop

Yuri Shimane, 2021/07/06

In [1]:
using LinearAlgebra
using DifferentialEquations

using Dates
using JSON
using Plots

In [27]:
plotly()

┌ Info: For saving to png with the Plotly backend ORCA has to be installed.
└ @ Plots C:\Users\yurio\.julia\packages\Plots\cc8wh\src\backends.jl:363


Plots.PlotlyBackend()

In [3]:
include("../R3BP/src/R3BP.jl")

Main.R3BP

In [4]:
function get_initial_condition(mu::Float64, rp::Float64, ra::Float64, theta::Float64, m::Int=2, velocity_dir::String="positive")
    # compute e
    e = (ra-rp)/(ra+rp)
    # compute v
    if m ==2
        v = sqrt( mu*(1+e)/rp )
    elseif m ==1
        v = sqrt( (1-mu)*(1+e)/rp ) 
    end
    # construct state
    if m==2
        x = (1 - mu) + rp*cos(theta)
    elseif m==1
        x = (-mu) + rp*cos(theta)   # FIXME?
    end
    y  =  rp*sin(theta)
    if velocity_dir=="positive"
        vx =  rp*sin(theta) - v*sin(theta)
        vy = -rp*cos(theta) + v*cos(theta)
    elseif velocity_dir=="negative"
        vx =  rp*sin(theta) + v*sin(theta)
        vy = -rp*cos(theta) - v*cos(theta)
    end
    return [ x, y, vx, vy ]
end

get_initial_condition (generic function with 3 methods)

In [16]:
function check_apsis(sv, mu::Float64)
    return dot([sv[1] - (1-mu), sv[2]], sv[3:4])
end

check_apsis (generic function with 1 method)

In [5]:
# set-up Sun-Earth system
params = R3BP.get_cr3bp_param(10, 399)
mu = params.mu
println("mu: $mu")

mu: 3.003480593992992e-6


In [47]:
# ---------- callbacks ---------- #
affect!(integrator) = terminate!(integrator)

# callback event based on having perigee around 0.9~1.1 * sma of the moon
moon_sma = 384748.0 / params.lstar
function condition_perigee_radius(u,t,integrator)
    r_local = sqrt((u[1] - (1-mu))^2 + u[2]^2)
    if 0.9moon_sma < r_local < 1.1moon_sma
        return check_apsis(u, mu)  # when hitting apsis
    else
        return NaN
    end
end
cb1 = ContinuousCallback(condition_perigee_radius, affect!, abstol=1.0e-12,reltol=1.0e-12)

# callback event based on intersecting with n*earth radius with vr<0
nr_earth = 10*6378.0 / params.lstar
function condition_nearth_Intersect(u,t,integrator)
    vr_sign = check_apsis(u, mu)
    if vr_sign < 0.0
        r_local = sqrt((u[1] - (1-mu))^2 + u[2]^2)
        return r_local - nr_earth
    else
        return NaN
    end
end
cb2 = ContinuousCallback(condition_nearth_Intersect, affect!, abstol=1.0e-12,reltol=1.0e-12)

# callback event when leaving n*earth_SOI
nearth_soi = 3*0.929e6 / params.lstar
function condiction_leave_nSOI(u,t,integrator)
    r_local = sqrt((u[1] - (1-mu))^2 + u[2]^2)
    return nearth_soi - r_local
end
cb3 = ContinuousCallback(condiction_leave_nSOI, affect!, abstol=1.0e-12,reltol=1.0e-12)

# create call back set
cbs = CallbackSet(cb1, cb2, cb3)

CallbackSet{Tuple{ContinuousCallback{typeof(condition_perigee_radius), typeof(affect!), typeof(affect!), typeof(DiffEqBase.INITIALIZE_DEFAULT), typeof(DiffEqBase.FINALIZE_DEFAULT), Float64, Float64, Nothing, Int64}, ContinuousCallback{typeof(condition_nearth_Intersect), typeof(affect!), typeof(affect!), typeof(DiffEqBase.INITIALIZE_DEFAULT), typeof(DiffEqBase.FINALIZE_DEFAULT), Float64, Float64, Nothing, Int64}, ContinuousCallback{typeof(condiction_leave_nSOI), typeof(affect!), typeof(affect!), typeof(DiffEqBase.INITIALIZE_DEFAULT), typeof(DiffEqBase.FINALIZE_DEFAULT), Float64, Float64, Nothing, Int64}}, Tuple{}}((ContinuousCallback{typeof(condition_perigee_radius), typeof(affect!), typeof(affect!), typeof(DiffEqBase.INITIALIZE_DEFAULT), typeof(DiffEqBase.FINALIZE_DEFAULT), Float64, Float64, Nothing, Int64}(condition_perigee_radius, affect!, affect!, DiffEqBase.INITIALIZE_DEFAULT, DiffEqBase.FINALIZE_DEFAULT, nothing, DiffEqBase.LeftRootFind, 10, Bool[1, 1], 1, 1.0e-12, 1.0e-12), Conti

In [33]:
# ---------- setup initial guess ---------- #
# perigee km
rp = (6378.0 + 185.0)/params.lstar

# setup storage
x0s = []
tfs = []
sim_info = []
n = 180
n_ra = 20

thetas = LinRange(0.0, 2π, n+1)[1:end-1]
ras = LinRange(0.8e6/params.lstar, 1.6e6/params.lstar, n_ra)

for ra in ras
    sma = (rp+ra)/2
    period = period = 2π*sqrt((sma*params.lstar)^3/R3BP.get_gm("399")[1]) / params.tstar
    for theta_iter in thetas
        push!(x0s, get_initial_condition(mu, rp, ra, theta_iter))
        push!(tfs, period)
        push!(sim_info, Dict("theta"=>theta_iter, "rp"=>rp, "ra"=>ra))
    end
end

# check number of initial guesses
nic = length(x0s)
ntf = length(tfs)
println("Using $nic - $ntf initial conditions")

Using 3600 - 3600 initial conditions


In [34]:
# ---------- thruster parameter ---------- #
mstar = 4100.0       # 4100 in kg for bepi-colombo
tmax_newtons = 0.4   # N, used to be 0.4
tmax = tmax_newtons *(1/mstar)*(params.tstar^2/(1e3*params.lstar))
isp = 3500.0   # seconds
mdot = tmax_newtons/(isp*9.80665) *(params.tstar/mstar)
println("tmax: $tmax")
println("mdot: $mdot")

tmax: 0.016452291023524804
mdot: 0.014276716503245912


In [35]:
# ---------- setup on ODE problem ---------- #
# tolerance of ODE problem
reltol = 1.0e-13
abstol = 1.0e-13

1.0e-13

In [36]:
# set-up initial problem
out_term = []

#for (idx_ensemble, τ_iter) in enumerate([-1.0, 1.0])
#println("\nEnsemble sim # $idx_ensemble ..... using τ = $τ_iter")
#τ = -1.0

τ_iter = -1.0
p = (mu, τ_iter, mdot, tmax)
m0 = 1.0
prob = ODEProblem(R3BP.rhs_pcr3bp_thrust_m1dir!, vcat(x0s[1], m0), (0.0, 2.0*tfs[1]), p)

# ---------- ensemble simulation ---------- #
function prob_func(prob, i, repeat)
    print("\rproblem # $i / $nic")
    remake(prob, u0=vcat(x0s[i], m0), tspan=(0.0, 2.0tfs[i]))
end
ensemble_prob = EnsembleProblem(prob, prob_func=prob_func)
sim = solve(ensemble_prob, Tsit5(), EnsembleThreads(), 
    trajectories=length(x0s), callback=cbs, reltol=reltol, abstol=abstol);

# ---------- post-process result ---------- #
# create storage for terminated solutions
for (idx, sol) in enumerate(sim)
    if sol.retcode == :Terminated
        # check which event terminated the propagation
        if isnan(condition_perigee_radius(sol.u[end], 0.0, 0.0)) == false
            push!(out_term, Dict(
                "x0" => sol.u[1],
                "xf" => sol.u[end],
                "tf" => sol.t[end],
                "theta" => sim_info[idx]["theta"],
                "rp" => sim_info[idx]["rp"],
                "ra" => sim_info[idx]["ra"],
                "τ" => τ_iter,
                "p" => p,
                "m0" => m0,
                "mf" => sol.u[end][end],
                "mstar" => mstar,
                "isp" => isp, 
                "mdot" => mdot,
                "tmax" => tmax,
                "mstar" => mstar,
            ))
        end
    end
end

n_found = length(out_term)
println("\nFound $n_found solutions!")

problem # 3600 / 3600
Found 26 solutions!


In [37]:
# # export data
# timestamp = Dates.format(Dates.now(), "yyyymmdd_HHMM")
# flename = "se_enhanced_" * timestamp
# flepath = "./let-SunEarthSystem-data/" * flename * ".json"
# open(flepath,"w") do f
#     JSON.print(f, out_term)
# end
# print("Save data at: ")
# println(flepath)

In [41]:
#idx_slows = [1576, 2201, 2291, 2718, 2767, 3177, 3480]   # for 360 - 20
idx_slows = [303, 776, 962, 1148, 1389, 1488, 1576, 1674, 1815, 2004, 2134, 2233, 2657, 
             2804, 2901, 3031, 3130, 3324, 3475, 3553, ];   # for 180 - 20

In [42]:
function plot_circle(radius, x, y, n=50)
    circle = zeros(2,n)
    thetas = LinRange(0.0, 2π, n)
    for i = 1:n
        circle[1,i] = radius*cos(thetas[i]) + x
        circle[2,i] = radius*sin(thetas[i]) + y
    end
    return circle
end

function list_to_plotarray_3d(sollist, state_idx_x, state_idx_y, state_idx_z)
    x_plot, y_plot, z_plot = [], [], []
    for k = 1:length(sollist)
        push!(x_plot, sollist[k][state_idx_x])
        push!(y_plot, sollist[k][state_idx_y])
        push!(z_plot, sollist[k][state_idx_z])
    end
    return x_plot, y_plot, z_plot
end

list_to_plotarray_3d (generic function with 1 method)

In [43]:
# get moon orbit for plotting
moon_sma = 384748.0 / params.lstar
moon_orbit = plot_circle(moon_sma, 1.0-mu, 0.0)
lp = R3BP.lagrangePoints(mu)

5×6 Matrix{Float64}:
  0.990027   0.0       0.0  0.0  0.0  0.0
  1.01003    0.0       0.0  0.0  0.0  0.0
 -1.0        0.0       0.0  0.0  0.0  0.0
  0.499997   0.866025  0.0  0.0  0.0  0.0
  0.499997  -0.866025  0.0  0.0  0.0  0.0

In [44]:
ptest = plot(flip=false, aspect_ratio=:equal, size=(900, 500), title="Sun-Earth PCR3BP, enhanced", 
             colorbar_title="tof, day", xlabel="x,km", ylabel="y, km", frame_style=:box)
scale = params.lstar

# Earth
scatter!(ptest, [1.0-mu]*scale, [0.0]*scale, marker=(:circle, 4.5, 3.0), c=:blue, label="Earth")

# Lagrange-points
scatter!(ptest, [lp[1,1]*scale], [lp[1,2]], marker=(:cross, 2.5, 3.0), c=:black, label="L1")
scatter!(ptest, [lp[2,1]*scale], [lp[2,2]], marker=(:xcross, 2.5, 3.0), c=:black, label="L2")

# moon orbit
plot!(ptest, moon_orbit[1,:]*scale, moon_orbit[2,:]*scale, c=:black, lw=1.2, label="Moon orbit")

# color map for trajectory
tofs = []
for sol in sim
    push!(tofs, sol.t[end]*params.tstar/86400.0)
end
segc = LinRange(minimum(tofs), maximum(tofs), length(sim))

# # propagated trajectories
# n_terminated = 0
# for (idx,sol) in enumerate(sim)
#     x_plot, y_plot, _ = list_to_plotarray_3d(sol, 1,2,3)
#     #plot!(ptest, x_plot*scale, y_plot*scale, c=:orangered, label=false, lw=0.75)
#     plot!(ptest, x_plot*scale, y_plot*scale, line_z=segc[idx], c=cgrad(:viridis), label=false, lw=0.75)
# end

# slower trajectory
for idx_slow in idx_slows
    x_plot, y_plot, _ = list_to_plotarray_3d(sim[idx_slow], 1,2,3)
    plot!(ptest, x_plot*scale, y_plot*scale, c=:orangered, label=false, lw=0.75)
end

ptest

<!DOCTYPE html>
 
 
 Plots.jl